In [4]:
from tkinter import messagebox, scrolledtext, filedialog, font
import requests
from bs4 import BeautifulSoup
from collections import Counter
import re
import tkinter as tk
from urllib.parse import urljoin

In [5]:


def analyze_images_and_links(soup, base_url):
    images = soup.find_all('img')
    links = soup.find_all('a')
    
    image_report = "Image Analysis Report:\n"
    for i, img in enumerate(images, 1):
        alt_text = img.get('alt', 'No Alt Text')
        image_report += f"Image {i}: Alt Text - {alt_text}\n"
    
    link_report = "Link Analysis Report:\n"
    broken_links = 0
    for i, link in enumerate(links, 1):
        href = link.get('href', '')
        full_url = urljoin(base_url, href)  # Convert relative URLs to absolute
        try:
            if full_url.startswith('http'):
                response = requests.head(full_url, allow_redirects=True, timeout=5)
                if response.status_code >= 400:
                    broken_links += 1
                    link_report += f"Link {i}: {full_url} (Broken Link)\n"
        except requests.RequestException:
            link_report += f"Link {i}: {full_url} (Potential Broken Link)\n"
    
    return image_report + "\n" + link_report + f"\nTotal Broken Links Found: {broken_links}"


In [6]:
def fetch_and_analyze_seo(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        text = soup.get_text(separator=' ')
        words = re.findall(r'\b\w+\b', text.lower())
        word_count = Counter(words)
        most_common_words = word_count.most_common(10)

        title = soup.find('title').text if soup.find('title') else "No Title"
        meta_description = soup.find('meta', attrs={'name': 'description'})
        meta_description_content = meta_description['content'] if meta_description else "No Meta Description"

        image_and_link_report = analyze_images_and_links(soup, url)

        report = f"SEO Analysis Report for {url}\n"
        report += f"Title: {title} ({len(title)} characters)\n"
        report += f"Meta Description: {meta_description_content} ({len(meta_description_content)} characters)\n"
        report += f"Most Common Words: {', '.join([f'{word} ({count})' for word, count in most_common_words])}\n"
        report += image_and_link_report
        report += "\nFull Text of the Page:\n" + text

        return report
    except requests.RequestException as e:
        return f"Error fetching URL: {e}"
    except Exception as e:
        return f"An error occurred: {e}"


In [7]:
def submit_action():
    url = url_entry.get()
    if not url:
        messagebox.showerror("Error", "Please enter a URL")
        return
    result = fetch_and_analyze_seo(url)
    report_text.delete(1.0, tk.END)
    report_text.insert(tk.END, result)

In [8]:
def save_report():
    report_data = report_text.get(1.0, tk.END)
    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
    if file_path:
        with open(file_path, 'w') as file:
            file.write(report_data)
        messagebox.showinfo("Save Report", "The report was saved successfully!")

In [9]:
root = tk.Tk()
root.title("SEO Analyzer")

default_font = font.nametofont("TkDefaultFont")
default_font.configure(family="Helvetica")

tk.Label(root, text="Enter URL:").pack()
url_entry = tk.Entry(root, width=50)
url_entry.pack()

submit_button = tk.Button(root, text="Analyze SEO", command=submit_action)
submit_button.pack()

report_text = scrolledtext.ScrolledText(root, height=20)
report_text.pack()

save_button = tk.Button(root, text="Save Report", command=save_report)
save_button.pack()

root.mainloop()